# 이번 분석에서 진행해야 할 것

1. JOIN_DATE를 카테고리화할 지에 대한 분석 
    - 기본 생각은 가입년도 별로 카테고리화 할 예정
    - 가입년도별 COUNT와의 관계를 좀 더 세밀하게 분석할 필요가 있음
    
    
2. DATE를 어떻게 활용할 지에 대한 분석
    - MONTH로 구분하자
    - 주별로 구분할 수도 있지만... 의미가 있을까?
    - 흐름 추세를 시각화하여 생각해보자.
    
    
3. 교통량 데이터에 대한 분석
    - 유입, 주중 오전 8시, 주말과 공휴일 오후 1시를 기준으로 확인하자 (퇴근 시간대 심야권도 생각해야 하나?)
    - 

4. Feature Engineering

In [13]:
import os 
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Malgun Gothic'  # (Windows 용) 한글 출력을 위한 글꼴 설정

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

import warnings
warnings.filterwarnings("ignore")

# 랜덤시드 통일
np.random.seed(42)

# set
pd.set_option('display.max_columns',100) # display 시 최대로 보이는 컬럼 개수
pd.set_option('display.max_rows',1000) # display 시 최대로 보이는 행의 개수
pd.set_option('display.width',600) 
pd.set_option('precision', 4) # 소수점 설정

In [2]:
def get_font_family():
    """
    시스템 환경에 따른 기본 폰트명을 반환하는 함수
    """
    import platform
    system_name = platform.system()
    # colab 사용자는 system_name이 'Linux'로 확인

    if system_name == "Darwin" :
        font_family = "AppleGothic"
    elif system_name == "Windows":
        font_family = "Malgun Gothic"
    else:
        # Linux
        # colab에서는 runtime을 <꼭> 재시작 해야함.
        # 런타임을 재시작 하지 않고 폰트 설치를 하면 기본 설정 폰트가 로드되어 한글이 깨짐.
        !apt-get update -qq
        !apt-get install fonts-nanum -qq  > /dev/null

        import matplotlib.font_manager as fm

        fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
        font = fm.FontProperties(fname=fontpath, size=9)
        fm._rebuild()
        font_family = "NanumBarunGothic"
    return font_family

In [3]:
# 시각화를 위한 폰트설정
# 위에서 만든 함수를 통해 시스템 폰트를 불러와서 font_family 라는 변수에 할당.
a = get_font_family()
# 폰트설정
import matplotlib.pyplot as plt 
plt.rc("font", family = a)
# 마이너스폰트 설정
plt.rc("axes", unicode_minus=False)
# ggplot으로 그래프 스타일 설정 / 개인 자유
plt.style.use("ggplot")

In [22]:
# 데이터 확인
os.listdir('data')

['2020교통량통합.xlsx',
 'holiday.csv',
 'metro.csv',
 '국가공휴일.xlsx',
 '디지털 스킬셋 기술과제.docx',
 '서울시_기상데이터.csv',
 '실전db.csv',
 '실전db_holiday.csv',
 '지하철노선위경도정보3.xlsx']

In [24]:
# 데이터프레임 불러오기
df = pd.read_csv("./data/실전db.csv")
# traffic_df = pd.read_excel("./data/2020교통량통합.xlsx")

In [51]:
df.columns

Index(['USER_ID', 'JOIN_DATE', 'D_TYPE', 'STORE_ID', 'GOODS_TYPE', 'DATE', 'COUNT', 'AD1'], dtype='object')

In [76]:
seoul = """1.종로구, JR

2. 중구, J

3.용산구, YO

4 성동구,SOD

 5광진구,GJ

 6동대문구, DM

7.중랑구, JRR

8.성북구, SB

9.강북구, GB

10.도봉구, DB

11. 노원구, NW

12.은평구, EP

13.서대문구, SD

14.마포구, MP

15.양천구, YC

16.강서구, GS

17.구로구, GR

18. 금천구, GHN

 19.영등포구,YD

 20.동작구, DJ

21. 관악구, GW

 22.서초구, SC

 23.강남구,GN

24. 송파구, SP

25.강동구 GD"""

import re

seoul = re.sub("[0-9.,가-힣 ]", "", seoul).split()
print(len(seoul))
print(seoul)

25
['JR', 'J', 'YO', 'SOD', 'GJ', 'DM', 'JRR', 'SB', 'GB', 'DB', 'NW', 'EP', 'SD', 'MP', 'YC', 'GS', 'GR', 'GHN', 'YD', 'DJ', 'GW', 'SC', 'GN', 'SP', 'GD']


In [77]:
ad1_list = sorted(df['AD1'].unique())

seoul_list = []
out_of_seoul = []

for k in ad1_list :
    new_df = df[df['AD1']==k]
    
    if k in seoul :
        seoul_list.append(new_df['STORE_ID'].nunique())
    else :
        out_of_seoul.append(new_df['STORE_ID'].nunique())
     

print(len(seoul_list))
print(len(out_of_seoul))

25
60


In [80]:
print(sum(seoul_list) / 25)
print(sum(out_of_seoul)/60)


28.64
5.75


# (1) JOIN_DATE

In [25]:
# 카피해서 사용
df_join = df.copy()

# 년도별 컬럼 생성
df_join.JOIN_DATE = pd.to_datetime(df_join.JOIN_DATE)
df_join.JOIN_DATE = df_join.JOIN_DATE.dt.to_period(freq="A")

In [26]:
display(df_join)

,USER_ID,JOIN_DATE,D_TYPE,STORE_ID,GOODS_TYPE,DATE,COUNT,AD1
0,2858,2014,AA,1892,A,2020-01-01,1,GN
1,5647,2014,BB,182009,A,2020-01-01,1,J
2,33314,2014,BB,82431,A,2020-01-01,1,SC
3,37001,2014,BB,725,C,2020-01-01,1,MP
4,37819,2014,AA,220691,C,2020-01-01,1,JRR
...,...,...,...,...,...,...,...,...
879266,1830551,2020,BB,219886,B,2020-12-31,1,GN
879267,1830570,2020,BB,82433,B,2020-12-31,1,CY
879268,1830580,2020,AA,92020,B,2020-12-31,1,JRR
879269,1830589,2020,BB,92437,B,2020-12-31,1,J


In [27]:
# 각 년도별로 이용자 수의 유니크 값 찾기
# 년도별 가입자 수의 데이터 행의 개수 확인
# 각 년도별 COUNT 값 파악

join_date_list = sorted(df_join['JOIN_DATE'].unique())

for year in join_date_list :
    year_df = df_join[df_join["JOIN_DATE"]==year]
    print(year,"년도 가입자 수 :",year_df["USER_ID"].nunique())
    print(year,"년도 가입자 전체 데이터 비율 :",(len(year_df)/len(df))*100,"%")
    print(year,"년도 가입자의 평균 COUNT :", year_df.COUNT.mean())
    print()
    print()

1970 년도 가입자 수 : 264
1970 년도 가입자 전체 데이터 비율 : 0.12430752293661454 %
1970 년도 가입자의 평균 COUNT : 1.0237877401646844


2013 년도 가입자 수 : 9
2013 년도 가입자 전체 데이터 비율 : 0.004208031426033612 %
2013 년도 가입자의 평균 COUNT : 1.027027027027027


2014 년도 가입자 수 : 549
2014 년도 가입자 전체 데이터 비율 : 0.336301322345443 %
2014 년도 가입자의 평균 COUNT : 1.0155563070679743


2015 년도 가입자 수 : 2679
2015 년도 가입자 전체 데이터 비율 : 1.6998172349594152 %
2015 년도 가입자의 평균 COUNT : 1.022614746420447


2016 년도 가입자 수 : 5796
2016 년도 가입자 전체 데이터 비율 : 3.719103666560139 %
2016 년도 가입자의 평균 COUNT : 1.0181645821228709


2017 년도 가입자 수 : 13500
2017 년도 가입자 전체 데이터 비율 : 9.091395030656077 %
2017 년도 가입자의 평균 COUNT : 1.0177387475293351


2018 년도 가입자 수 : 22936
2018 년도 가입자 전체 데이터 비율 : 15.981762164338411 %
2018 년도 가입자의 평균 COUNT : 1.0724934708197236


2019 년도 가입자 수 : 42124
2019 년도 가입자 전체 데이터 비율 : 29.297338363257747 %
2019 년도 가입자의 평균 COUNT : 1.0189826981828627


2020 년도 가입자 수 : 77568
2020 년도 가입자 전체 데이터 비율 : 39.74576666352012 %
2020 년도 가입자의 평균 COUNT : 1.0230861897771788




In [37]:
# 2018년도에 특정 가입일자에 이상치가 많았다?!
df_join_2018 = df_join[df_join['JOIN_DATE']==join_date_list[-3]]

display(df_join_2018['COUNT'].value_counts())

[Period('1970', 'A-DEC'), Period('2013', 'A-DEC'), Period('2014', 'A-DEC'), Period('2015', 'A-DEC'), Period('2016', 'A-DEC'), Period('2017', 'A-DEC'), Period('2018', 'A-DEC'), Period('2019', 'A-DEC'), Period('2020', 'A-DEC')]


1     137815
2       2162
3        126
4         30
16        21
12        19
14        19
28        17
13        16
9         15
15        15
5         14
17        14
18        14
21        14
25        14
7         13
23        13
11        12
19        12
10        10
8         10
32         9
6          9
26         9
22         8
34         7
27         7
24         7
31         7
30         7
29         6
36         5
20         5
40         4
35         3
46         3
49         3
38         3
39         3
37         3
33         2
42         2
50         2
43         2
47         2
55         1
58         1
53         1
48         1
51         1
59         1
41         1
44         1
45         1
61         1
Name: COUNT, dtype: int64

In [46]:
df_join_2018_count = df_join_2018.query("COUNT > 5")


In [48]:
print(len(df_join_2018_count))
display(df_join_2018_count.head())

376


,USER_ID,JOIN_DATE,D_TYPE,STORE_ID,GOODS_TYPE,DATE,COUNT,AD1
372,999665,2018,CC,82399,A,2020-01-01,6,JRR
1527,999665,2018,CC,104988,A,2020-01-02,11,GN
3121,999665,2018,CC,181832,A,2020-01-03,7,SC
5034,999665,2018,CC,109223,A,2020-01-04,28,MP
6919,999665,2018,CC,104916,A,2020-01-05,14,GN


In [49]:
df_join_2018_count['DATE'].unique()

array(['2020-01-01', '2020-01-02', '2020-01-03', '2020-01-04',
       '2020-01-05', '2020-01-06', '2020-01-07', '2020-01-08',
       '2020-01-09', '2020-01-10', '2020-01-11', '2020-01-12',
       '2020-01-13', '2020-01-14', '2020-01-15', '2020-01-16',
       '2020-01-17', '2020-01-18', '2020-01-19', '2020-01-20',
       '2020-01-21', '2020-01-22', '2020-01-23', '2020-01-24',
       '2020-01-25', '2020-01-26', '2020-01-27', '2020-01-28',
       '2020-01-30', '2020-01-31', '2020-02-01', '2020-02-02',
       '2020-02-03', '2020-02-04', '2020-02-05', '2020-02-06',
       '2020-02-07', '2020-02-08', '2020-02-09', '2020-02-10',
       '2020-02-11', '2020-02-12', '2020-02-13', '2020-02-14',
       '2020-02-15', '2020-02-16', '2020-02-17', '2020-02-20',
       '2020-02-21', '2020-02-22', '2020-02-24', '2020-02-25',
       '2020-02-26', '2020-02-27', '2020-02-28', '2020-02-29',
       '2020-03-01', '2020-03-02', '2020-03-03', '2020-03-04',
       '2020-03-05', '2020-03-06', '2020-03-07', '2020-

# (2) DATE

# (3) 교통량 데이터

In [14]:
display(traffic_df.head())

,DATE,지점명,지점번호,방향,구분,0시,1시,2시,3시,4시,5시,6시,7시,8시,9시,10시,11시,12시,13시,14시,15시,16시,17시,18시,19시,20시,21시,22시,23시
0,2020-01-01,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,712.0,645.0,437.0,309.0,290.0,338.0,504.0,574.0,853.0,826.0,1051.0,1302.0,1362.0,1376.0,1472.0,1416.0,1483.0,1329.0,1157.0,1014.0,954.0,849.0,780.0,480.0
1,2020-01-02,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,315.0,222.0,186.0,165.0,266.0,716.0,1561.0,2530.0,2524.0,2092.0,1904.0,1850.0,1730.0,1722.0,1792.0,1897.0,1842.0,2061.0,1994.0,1443.0,1233.0,1165.0,1094.0,852.0
2,2020-01-03,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,632.0,457.0,295.0,236.0,279.0,681.0,1385.0,2323.0,2544.0,2115.0,2018.0,2014.0,1693.0,1707.0,2004.0,1929.0,2049.0,2140.0,2178.0,1654.0,1356.0,1260.0,1253.0,941.0
3,2020-01-04,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,740.0,518.0,388.0,331.0,330.0,492.0,798.0,970.0,1369.0,1639.0,1828.0,2025.0,1899.0,1876.0,1837.0,1788.0,1588.0,1669.0,1530.0,1222.0,1143.0,1089.0,1039.0,791.0
4,2020-01-05,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,533.0,424.0,297.0,230.0,209.0,302.0,525.0,731.0,1098.0,1434.0,1578.0,1567.0,1605.0,1679.0,1634.0,1673.0,1494.0,1429.0,1288.0,1035.0,987.0,884.0,803.0,564.0


In [15]:
# 데이터 형태 확인
# 데이터 내부 살펴보기 함수
def search_data(df):

    print("데이터 살펴보기 :")
    display(df.describe())
    print()
    print()    
    
    print("데이터 정보 확인 :")
    print(df.info())
    print()
    print()
        
    print("데이터 형태 확인 :", df.shape)
    print()
    print()
    
    print("결측치 확인 :")
    print(df.isna().sum())
    print()
    print()
    
    print("유니크값 확인 :")
    for col in df.columns:
        print(f"column : {col}")
        print(f"The number of unique : {df[col].nunique()}")
        print()


In [16]:
search_data(traffic_df)

데이터 살펴보기 :


,0시,1시,2시,3시,4시,5시,6시,7시,8시,9시,10시,11시,12시,13시,14시,15시,16시,17시,18시,19시,20시,21시,22시,23시
count,91985.0000,91962.0000,91960.0000,91979.0000,91999.0000,92036.0000,92079.0000,92095.0000,92081.0000,92110.0000,92144.0000,92100.0000,92067.0000,92032.0000,92053.0000,92066.0000,92093.0000,92067.0000,92079.0000,92034.000,92040.0000,92023.0000,91824.0000,91698.0000
mean,738.5552,529.3702,395.7212,334.5236,414.2912,834.1101,1428.2915,1802.4209,1936.5287,1941.2440,1938.2767,1934.1192,1905.2911,1967.7212,1993.5412,2016.8486,2063.2317,2084.8786,1993.7741,1806.331,1654.0213,1578.7286,1341.8157,1004.1664
std,634.1311,460.3107,345.1625,290.9829,370.3570,856.6778,1328.0682,1367.6322,1336.2875,1291.1743,1284.2317,1282.0119,1278.4211,1293.8471,1292.5605,1299.1664,1307.3313,1279.3342,1231.2663,1188.160,1187.3743,1213.1589,1090.3519,853.5791
min,1.0000,0.0000,0.0000,0.0000,1.0000,4.0000,6.0000,7.0000,1.0000,7.0000,10.0000,12.0000,11.0000,12.0000,12.0000,14.0000,11.0000,9.0000,7.0000,9.000,7.0000,8.0000,6.0000,7.0000
25%,330.0000,228.0000,167.0000,141.0000,181.0000,325.0000,557.0000,791.0000,960.0000,1039.0000,1076.0000,1080.0000,1069.0000,1115.0000,1137.0000,1144.0000,1173.0000,1194.0000,1133.0000,985.000,852.0000,784.0000,634.0000,456.0000
50%,556.0000,397.0000,297.0000,247.0000,303.0000,546.0000,997.0000,1474.0000,1637.0000,1650.0000,1621.0000,1620.0000,1586.0000,1662.0000,1693.0000,1707.0000,1748.0000,1778.0000,1714.0000,1513.000,1338.0000,1252.0000,1025.0000,752.0000
75%,932.0000,680.0000,512.0000,427.0000,500.0000,975.0000,1787.0000,2401.0000,2563.0000,2517.0000,2455.0000,2397.0000,2329.0000,2422.0000,2463.0000,2512.0000,2603.0000,2648.0000,2547.0000,2331.000,2106.0000,1995.0000,1695.0000,1260.0000
max,5717.0000,5274.0000,3359.0000,3752.0000,4328.0000,6924.0000,7896.0000,8324.0000,7931.0000,7549.0000,7592.0000,7824.0000,8021.0000,8054.0000,8138.0000,8330.0000,8304.0000,8442.0000,7798.0000,7576.000,7927.0000,7855.0000,7806.0000,7548.0000




데이터 정보 확인 :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98820 entries, 0 to 98819
Data columns (total 29 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   DATE    98820 non-null  datetime64[ns]
 1   지점명     98820 non-null  object        
 2   지점번호    98820 non-null  object        
 3   방향      98820 non-null  object        
 4   구분      98820 non-null  object        
 5   0시      91985 non-null  float64       
 6   1시      91962 non-null  float64       
 7   2시      91960 non-null  float64       
 8   3시      91979 non-null  float64       
 9   4시      91999 non-null  float64       
 10  5시      92036 non-null  float64       
 11  6시      92079 non-null  float64       
 12  7시      92095 non-null  float64       
 13  8시      92081 non-null  float64       
 14  9시      92110 non-null  float64       
 15  10시     92144 non-null  float64       
 16  11시     92100 non-null  float64       
 17  12시     92067 non-null  float64     

문제는 데이터 내 결측치가 너무 많다는 것         
공휴일,주말이면 공휴일 평균과 주말 평균으로 보간        
주중이면 주중 평균으로 보간            
기상데이터에서는 주중 7시 / 주말과 공휴일 12시를 기준으로 했다. ==> 기상데이터도 공휴일 고려하는 부분을 추가할 것        


In [17]:
# 우선 유입, 유출 외 다른 값이 뭔지 확인 좀
print(traffic_df['방향'].unique()) 

# 다른 하나는 단순히 NULL값 
# 방향으로 볼 때 유입은 서울 내로 진입하는 것으로 판단
# 주차장은 서울 내 이용을 가정?
# 근데 서울 내 주차장만 있는 것은 아니라서 현재 이를 구분해낼 수 있는 방법 자체가 없음

['유입' '유출' ' ']


In [ ]:
# 기본 컬럼에 대해서 생각해보자

# 유입 유출에 대한 구분 / 